<a href="https://colab.research.google.com/github/bradpatton/HealthcareAI/blob/main/AgentBasedHealthcare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [167]:
!pip install --q crewai
!pip install --q -U duckduckgo-search
!pip install --q langchain_openai
!pip install --q xmltodict
!pip install --q 'crewai[tools]'


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.2/342.2 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [245]:
import os
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew, Process
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import load_tools
from langchain.agents import AgentType, Tool, initialize_agent
from google.colab import userdata
from langchain_community.tools.pubmed.tool import PubmedQueryRun
from crewai_tools import SerperDevTool
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.tools import DuckDuckGoSearchRun
med_tool = PubmedQueryRun()
search_tool = DuckDuckGoSearchRun()
#search_tool = SerperDevTool(serper_api_key=userdata.get('SERPER_API_KEY'))
#search_tool = load_tools(["google-serper"],serper_api_key=userdata.get('SERPER_API_KEY')) #GoogleSerperAPIWrapper(serper_api_key=userdata.get('OPENAI_API_KEY'))

In [246]:
llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0, api_key=userdata.get('OPENAI_API_KEY'))

In [247]:
writer = Agent(
  role='Sympathetic Professional Medical Writer',
  goal='Explain with a sympathetic tone and age-based language to a newly diagnosed patient the nature of there ailment,impact and lifestyle changes that they will encounter with their new diagnosis ##Exclude medical lingo or terminology ## Draft must be reviewed by a medical professional for accuracy',
  backstory="""You work in a hospital that supports patients from pediatric through to geriatric ages. You are known for turning a complex a diagnosis into relatable and sympathetic writing that clearly explains the nature of ones disease to a patient. """,
  verbose=True,
  allow_delegation=True,
  llm = llm,
  tools=[
        med_tool
      ]
)

In [248]:
coach = Agent(
  role='Medical Coach',
  goal='Create a lifestyle plan for a newly diagnosed patient containing an age-based excercise and mobility plan,  ',
  backstory="""You are a renowned Medical Coach, known for boosting your patients quality of life. You are engaging and build lifestyle plans that are fun and improve the patients quality of life.""",
  verbose=True,
  allow_delegation=True,
  llm = llm,
  tools=[
        search_tool
      ]
)

In [249]:
chef = Agent(
  role='Dietician and Chef',
  goal='Create a report meal plan and recipe book for the patient based on clinical diagnosis',
  backstory="""You are a renowned chef, known for
  creative dishes that make hospital food delicious. You can search the internet for reccomendations of nutrients and foods that benefit the condition as well as search for recipes that include these nutrients and foods.""",
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools =[search_tool]
)

In [250]:
doctor = Agent(
  role='Medical Professional',
  goal='Advise and review infomration to confirm that it is medically sound and suggest revisions and edits if the infomration is nto medically correct',
  backstory="""You are a renowned medical expert known for your encyclopedic knowledge of biology and medicine.You can search the internet for medical infomration or pharmacological information""",
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools =[search_tool

      ]
)

In [251]:
task1 = Task(
  description="""Complete a guide for a newly diagnosed patient that is a 37 year old woman diagnosed with Triple-negative breast cancer""",
  expected_output="""A Detailed and individualized report on diagnosis in terms that the patient can understand. This should include a detailed sections with a description of the condition in age relevant terms and with sympathetic tone, the things that can be expected to change in life and the impact that may have, potential medications or procedures that may be prescribed with side effects or recovery procedures, and a detailed prognosis or receovery care plan. This should all be reviewed by a medical professional agent and finally edited by the writer for tone. """,
  agent=writer
)

In [252]:
task2 = Task(
  description="""Create a meal plan and recipe book for a patient diagnosed with Triple-negative breast cancer""",
  expected_output="""A individulized report containing a detailed description of foods, macros and nutrients and why they have a positive impact on the specific diagnosis or recovery. A complete Recipe Guide in Markdown each recipe should contain a description of each element from the meal plan that it showcases and a complete recipe with ingredients, measurements and complete cooking instructions""",
  agent=chef
)

In [253]:
task3 = Task(
  description="""Create a lifestyle plan for a patient diagnosed with Triple-negative breast cancer""",
  expected_output="""A Detailed lifestyle plan report in Markdown. This should contain a section on daily excercise routine, mobility excercises, and reccomendations on ways to improve the patients quality of life following the diagnosis """,
  agent=coach
)

In [254]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[writer, coach, chef, doctor],
  tasks=[task1, task2, task3],
  verbose=2,

)

In [255]:
result = crew.kickoff()

 [DEBUG]: == Working Agent: Sympathetic Professional Medical Writer
 [INFO]: == Starting Task: Complete a guide for a newly diagnosed patient that is a 37 year old woman diagnosed with Triple-negative breast cancer


> Entering new CrewAgentExecutor chain...
I need to gather comprehensive and accurate information about Triple-negative breast cancer to create a sympathetic and understandable guide for a 37-year-old woman who has just been diagnosed with this condition. I should start by understanding the basics of Triple-negative breast cancer, its impact, treatment options, and prognosis in a way that's devoid of complex medical jargon. This will help me draft a guide that's both informative and comforting. I'll need to consult with a medical professional to ensure the accuracy of the medical content before finalizing the tone and language for the patient.

Action: pub_med
Action Input: {"query": "Triple-negative breast cancer overview"} 

Published: 2024-04-16
Title: Comprehensive und

In [244]:
print(result)

# Osteochondritis Dissecans Lifestyle Plan

## Introduction
This lifestyle plan is designed to complement the Osteochondritis Dissecans Recovery Meal Plan and Recipe Book, focusing on exercises and mobility activities that support bone health, reduce inflammation, and aid in recovery. Tailored to different age groups, this plan aims to improve the quality of life for individuals diagnosed with Osteochondritis Dissecans (OCD).

## Daily Exercise Routine

### Children and Adolescents
- **Swimming (3-4 times a week):** Encourages cardiovascular health without stressing the joints.
- **Cycling (2-3 times a week):** Another low-impact activity that promotes leg strength and endurance.
- **Physical Therapy Exercises (Daily):** Specific exercises provided by a physical therapist to improve range of motion and strengthen muscles around the affected joint.

### Adults
- **Swimming/Water Aerobics (3-4 times a week):** Ideal for reducing joint stress while exercising.
- **Cycling or Walking (3-4 